# Long Sentence Resolution - Preprocess Splitting Datasets

# Documentation

## Datasets

Dataset | #examples before preprocessing | #examples after preprocessing | %examples kept after preprocessing
:- | :- | :- | :-
WikiSplit | 999944 | 298534 | 29.9%
BiSECT | 940259 | 197420 | 21.0%

## What is a word?

* A word is any token that is not any of the following:
  * bracket
  * punctuation
  * quote
  * space
  * currency

## Connect Google Drive
 
* Run the cell "Connect Google Drive" and on the popup window click "Connect to Google Drive".
 
* In the new page opened, choose an account (your account) and click Allow.

## Configuration parameters

* **Source minimum #words**: The number of words that a sentence should have to be considered a valid source sentence. All sentences that have less words than this threshold are dropped (default: 17).

* **Target maximum #words**: The number of words that a sentence should have to be considered a valid target sentence. All sentences that have more words than this threshold are dropped (default: 16).

# Installations

In [ ]:
!pip install datasets==2.5.2 \
             spacy==3.4.1 \
             sacremoses==0.0.53

# Imports

In [ ]:
import datetime, spacy, json

import pandas as pd

import ipywidgets as widgets

import plotly.express as px

import numpy as np

from typing import List, Dict

from pathlib import Path

from datasets import Dataset, concatenate_datasets, load_from_disk

from google.colab import drive

from spacy.tokens import Doc, Token, Span

from sacremoses import MosesDetokenizer

# Configuration parameters

In [ ]:
source_minimum_n_words_bounded_int_text = widgets.BoundedIntText(value=17, min=1, max=20, step=1)

target_maximum_n_words_bounded_int_text = widgets.BoundedIntText(value=16, min=1, max=20, step=1)

display(widgets.HBox([widgets.Label(value="Source minimum #words:"), source_minimum_n_words_bounded_int_text]))

display(widgets.HBox([widgets.Label(value="Target maximum #words:"), target_maximum_n_words_bounded_int_text]))

# Connect Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

# Create paths

In [ ]:
RAW_DATA_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Datasets/raw')

PREPROCESSED_DATA_DIRECTORY_PATH = Path('/content/drive/My Drive/Pfizer - Long Sentence Resolution/Datasets/preprocessed')

PREPROCESSED_DATA_DIRECTORY_PATH.mkdir(parents=True, exist_ok=True)

# Functions / Classes

In [ ]:
def preprocess_wiki_split_dataset(ds: Dataset) -> Dataset:
  md = MosesDetokenizer(lang='en')
  
  def map_operation(example: Dict) -> Dict:
    example["source"] = md.detokenize(example['source'].strip().split())

    example["target"] = md.detokenize(" ".join([ example['simple_sentence_1'].strip(), example['simple_sentence_2'].strip() ]).split())

    example["dataset"] = "WikiSplit"

    example["full_path"] = create_full_path_from_dataset_example(example)

    return example

  return ds.rename_columns({'complex_sentence': 'source'}).map(map_operation)

def preprocess_bisect_dataset(ds: Dataset) -> Dataset:
  md = MosesDetokenizer(lang='en')
  
  def map_operation(example: Dict) -> Dict:
    example["source"] = md.detokenize(example['source'].strip().split())

    example["target"] = md.detokenize(example['target'].strip().split())

    example["dataset"] = "BiSECT"

    example["full_path"] = create_full_path_from_dataset_example(example)

    return example

  return ds.map(map_operation)

def extract_n_words_from_document(doc: Doc) -> int:
  return sum(map(token_is_word, doc))

def extract_n_words_from_span(span: Span) -> int:
  return sum(map(token_is_word, span))

def extract_n_sentences(doc: Doc) -> int:
  return len(list(doc.sents))

def extract_sentence_n_words_from_document(document: Doc) -> List[int]:
  return list(map(extract_n_words_from_span, document.sents))

def token_is_word(token: Token) -> bool:
  return not any([ token.is_bracket,
                   token.is_punct,
                   token.is_currency,
                   token.is_quote,
                   token.is_space ]) # https://spacy.io/api/token#attributes

def create_full_path_from_dataset_example(example: Dict) -> str:
  full_path = example["dataset"]

  if 'subset' in example:
    full_path = full_path + f'>{example["subset"]}'

  if 'split' in example:
    full_path = full_path + f'>{example["split"]}'

  return full_path

# Load raw datasets

In [ ]:
%%time

wiki_split_dataset = load_from_disk(RAW_DATA_DIRECTORY_PATH / 'wiki_split')

wiki_split_dataset

In [ ]:
%%time

bisect_dataset = load_from_disk(RAW_DATA_DIRECTORY_PATH / 'bisect')

bisect_dataset

# Preprocess raw datasets

## Load models

In [ ]:
from spacy.lang.en import English

linguistics_model = English()

linguistics_model.add_pipe("sentencizer")

## Normalize raw datasets

In [ ]:
%%time

wiki_split_dataset = preprocess_wiki_split_dataset(wiki_split_dataset)

wiki_split_dataset

In [ ]:
%%time

bisect_dataset = preprocess_bisect_dataset(bisect_dataset)

bisect_dataset

## Merge raw datasets

In [ ]:
splitting_datasets_to_concatenate = [ ds.remove_columns(set(ds.column_names).difference(['source', 'target', 'dataset', 'split', 'full_path'])) for ds in [
  wiki_split_dataset,
  bisect_dataset
]]

splitting_dataset = concatenate_datasets(splitting_datasets_to_concatenate)

splitting_dataset

## Convert raw dataset to dataframe

In [ ]:
splitting_dataset_df = splitting_dataset.to_pandas()

In [ ]:
splitting_dataset_df

## Count #words and #sentences

In [ ]:
%%time

splitting_dataset_df[['source_n_sentences', 'source_n_words']] = pd.DataFrame(map(lambda o: { 'n_sentences' : extract_n_sentences(o), 'n_words': extract_n_words_from_document(o) }, linguistics_model.pipe(splitting_dataset_df['source'], batch_size=50)), index=splitting_dataset_df.index)

splitting_dataset_df['target_n_words_per_sentence'] = pd.Series(map(extract_sentence_n_words_from_document, linguistics_model.pipe(splitting_dataset_df['target'], batch_size=50)), index=splitting_dataset_df.index)

splitting_dataset_df['target_n_sentences'] = splitting_dataset_df['target_n_words_per_sentence'].apply(len)

## Statistics on raw datasets

### Datasets #examples and ratios

In [ ]:
splitting_dataset_df['dataset'].value_counts()

In [ ]:
np.round((splitting_dataset_df['dataset'].value_counts(normalize=True) * 100), 2).astype(str) + '%'

### Summary statistics on #words and #sentences per dataset

In [ ]:
splitting_dataset_df.dtypes

In [ ]:
splitting_dataset_df.groupby('dataset') [['source_n_words']].describe().apply(lambda s: s.apply('{0:.5f}'.format)).T

In [ ]:
splitting_dataset_df.groupby('dataset') [['source_n_sentences', 'target_n_sentences']].describe().apply(lambda s: s.apply('{0:.5f}'.format)).T

### Box plots on #words per dataset (20% sample is used)

In [ ]:
px.box(splitting_dataset_df.groupby('dataset', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42)), x="dataset", y="source_n_words")

## Filter source-target pairs

In [ ]:
splitting_dataset_df = splitting_dataset_df[splitting_dataset_df['source_n_sentences'] == 1]

In [ ]:
splitting_dataset_df = splitting_dataset_df[(splitting_dataset_df['source_n_words'] >= source_minimum_n_words_bounded_int_text.value) & (splitting_dataset_df['target_n_words_per_sentence'].apply(lambda o: all([x <= target_maximum_n_words_bounded_int_text.value for x in o])))]

In [ ]:
splitting_dataset_df.drop_duplicates(subset=['source', 'target'], inplace=True, ignore_index=True)

## Statistics on preprocessed datasets

### Datasets #examples and ratios

In [ ]:
splitting_dataset_df['dataset'].value_counts()

In [ ]:
np.round((splitting_dataset_df['dataset'].value_counts(normalize=True) * 100), 2).astype(str) + '%'

### Summary statistics on #words and #sentences per dataset

In [ ]:
splitting_dataset_df.dtypes

In [ ]:
splitting_dataset_df.groupby('dataset') [['source_n_words']].describe().apply(lambda s: s.apply('{0:.5f}'.format)).T

In [ ]:
splitting_dataset_df.groupby('dataset') [['source_n_sentences', 'target_n_sentences']].describe().apply(lambda s: s.apply('{0:.5f}'.format)).T

### Box plots on #words per dataset (20% sample is used)

In [ ]:
px.box(splitting_dataset_df.groupby('dataset', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=42)), x="dataset", y="source_n_words")

## Save preprocessed dataset

In [ ]:
preprocessed_splitting_dataset_df = splitting_dataset_df [['source', 'target', 'dataset', 'split', 'full_path']]

preprocessed_splitting_dataset_df

In [ ]:
preprocessed_dataset_directory_path = PREPROCESSED_DATA_DIRECTORY_PATH / f'splitting_dataset_{datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}'

In [ ]:
preprocessed_splitting_dataset = Dataset.from_pandas(preprocessed_splitting_dataset_df, preserve_index=False)

preprocessed_splitting_dataset.save_to_disk(preprocessed_dataset_directory_path)

with open(preprocessed_dataset_directory_path / 'config.json', 'w') as f: json.dump({
    "source_minimum_n_words" : source_minimum_n_words_bounded_int_text.value,
    "target_maximum_n_words" : target_maximum_n_words_bounded_int_text.value,
    "full_paths_in_preprocessed_data" : sorted(preprocessed_splitting_dataset.unique('full_path'))
}, f, sort_keys=True, indent=4)